# 1. Libraries

In [25]:
import pandas as pd
import numpy as np
import os
import datetime

# 2. Data load

In [26]:
merged = pd.read_csv('/home/nivelrios/documentos/Mburicao Project/data/external/sil_nivel_merged.csv')
picos = pd.read_csv('/home/nivelrios/documentos/Mburicao Project/data/external/peak_nivel_detection.csv')

# 3. Data extraction

In [27]:
merged['fecha'] = pd.to_datetime(merged['fecha'], errors='coerce')

picos['inicio'] = pd.to_datetime(picos['inicio'], errors='coerce')
picos['fin'] = pd.to_datetime(picos['fin'], errors='coerce')
picos['max_global_fecha'] = pd.to_datetime(picos['max_global_fecha'], errors='coerce')
picos['max_local_fecha'] = pd.to_datetime(picos['max_local_fecha'], errors='coerce')

In [28]:
if "fecha" in merged.columns:
    merged["fecha"] = pd.to_datetime(merged["fecha"], errors='coerce')
    merged.set_index("fecha", inplace=True)
else:
    merged.index = pd.to_datetime(merged.index, errors='coerce')
picos["max_global_fecha"] = pd.to_datetime(picos["max_global_fecha"], errors='coerce')

features_list = []

for idx, row in picos.iterrows():
    base_time = row["max_global_fecha"]
    global_peak = row["max_global"]
    all_dates_exist = True
    for delay in range(0, 121, 10):
        time_point = base_time - datetime.timedelta(minutes=delay)
        if time_point not in merged.index:
            all_dates_exist = False
            break
    if not all_dates_exist:
        continue
    feature_dict = {
        "fecha": base_time,
        "global_peak": global_peak
    }
    for delay in range(0, 81, 10):
        col_name = f"sil_{delay}"
        time_point = base_time - datetime.timedelta(minutes=delay)
        sil_value = merged.loc[time_point, "sil"]
        if isinstance(sil_value, pd.Series):
            sil_value = sil_value.iloc[0]
        feature_dict[col_name] = sil_value
    for accum_delay in range(0, 71, 10):
        col_accum = f"sil_accumulated_{accum_delay}"
        total = 0
        for d in range(accum_delay, 81, 10):
            total += feature_dict[f"sil_{d}"]
        feature_dict[col_accum] = round(total, 1)

    features_list.append(feature_dict)
data = pd.DataFrame(features_list)

# 4. Save dataframe

In [29]:
ruta = "/home/nivelrios/documentos/Mburicao Project/src/data"
os.makedirs(ruta, exist_ok=True)
archivo = os.path.join(ruta, "data_extraction.csv")
data.to_csv(archivo, index=False)